In [1]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [3]:
# Read in nasdaq data and extract symbols column
df = pd.read_csv("earnings_pricing.csv")
# Pull API key from .env file
EOD_API_KEY = config("EOD_API_KEY")
# Gather dates to iterate over
dates = df["date"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75955 entries, 0 to 75954
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              75955 non-null  object 
 1   symbol            75955 non-null  object 
 2   eps               75955 non-null  float64
 3   epsEstimated      75955 non-null  float64
 4   time              75955 non-null  object 
 5   open              75955 non-null  float64
 6   high              75955 non-null  float64
 7   low               75955 non-null  float64
 8   close             75955 non-null  float64
 9   adjClose          75955 non-null  float64
 10  volume            75955 non-null  int64  
 11  unadjustedVolume  75955 non-null  int64  
 12  change            75955 non-null  float64
 13  changePercent     75955 non-null  float64
 14  vwap              75955 non-null  float64
 15  changeOverTime    75955 non-null  float64
dtypes: float64(11), int64(2), object(3)
memo

In [4]:
df_sorted = df.sort_values(by='date', ascending=True)
print(df_sorted)

# We start on 1/10/12 so for period 5 we ill have data starting at 2012=01-01

          date symbol   eps  epsEstimated time       open       high  \
62611  1/10/12    SNX  1.37          1.14  amc  15.895000  16.055000   
45558  1/10/13    MSM  1.01          1.01  bmo  72.010002  74.150002   
62607  1/10/13    SNX  1.16          1.04  amc  17.825001  17.934999   
35539  1/10/13   INFY  0.10          0.09  bmo   5.371250   5.595000   
62591  1/10/17    SNX  2.57          2.09  amc  60.500000  61.575001   
...        ...    ...   ...           ...  ...        ...        ...   
53791   9/9/21   PLAY  1.07          0.57  amc  34.550000  36.100000   
58563   9/9/21   RSKD -1.41         -1.46  bmo  30.600000  32.320000   
19114   9/9/21    CXM -0.20         -0.21  amc  17.910000  19.780000   
5937    9/9/21    ASO  2.34          1.54  bmo  41.680000  45.470000   
64524   9/9/21   SUMO -0.24         -0.28  amc  20.010000  20.470000   

             low      close   adjClose    volume  unadjustedVolume  change  \
62611  15.800000  16.045000  15.014843    562200         

In [5]:
stock = df.symbol.unique()
print(stock)
print("Number of stocks: {}".format(len(stock)))

['A' 'AA' 'AAL' ... 'ZUMZ' 'ZUO' 'ZWS']
Number of stocks: 2628


In [6]:
start_date = '2011-10-01'
end_date = '2022-01-01'
func = 'wma'
period = 20

# For testing
sample_stock = ['AAPL', 'MSFT']

df = pd.DataFrame()
for idx, val in enumerate(stock):
    try:
        print("Working iteration: {}".format(idx))
        url = "https://eodhistoricaldata.com/api/technical/{}.US?order=d&fmt=json&from={}&to={}&function={}&period={}&api_token={}".format(val, start_date, end_date, func, period, EOD_API_KEY)
        res = get_jsonparsed_data(url)
        res_df = pd.DataFrame.from_records(res)
        # Insert symbol
        res_df.insert(1, "symbol", val)
        # Concat with main dataframe
        df = pd.concat([df, res_df])
    except Exception as e:
        print("Exception on {}: {}".format(idx, e))

Working iteration: 0
Working iteration: 1
Working iteration: 2
Working iteration: 3
Working iteration: 4
Working iteration: 5
Working iteration: 6
Working iteration: 7
Working iteration: 8
Working iteration: 9
Working iteration: 10
Working iteration: 11
Working iteration: 12
Working iteration: 13
Working iteration: 14
Working iteration: 15
Working iteration: 16
Working iteration: 17
Working iteration: 18
Working iteration: 19
Working iteration: 20
Working iteration: 21
Working iteration: 22
Working iteration: 23
Working iteration: 24
Working iteration: 25
Working iteration: 26
Working iteration: 27
Working iteration: 28
Working iteration: 29
Working iteration: 30
Working iteration: 31
Working iteration: 32
Working iteration: 33
Working iteration: 34
Working iteration: 35
Working iteration: 36
Working iteration: 37
Working iteration: 38
Working iteration: 39
Working iteration: 40
Working iteration: 41
Working iteration: 42
Working iteration: 43
Working iteration: 44
Working iteration: 4

In [7]:
df = df.rename(columns={"wma": "wma_20"})
print(df)

            date symbol   wma_20
0     2021-12-31      A  156.351
1     2021-12-30      A  155.890
2     2021-12-29      A  155.251
3     2021-12-28      A  154.587
4     2021-12-27      A  154.036
...          ...    ...      ...
2433  2012-05-02    ZWS   21.352
2434  2012-05-01    ZWS   21.313
2435  2012-04-30    ZWS   21.276
2436  2012-04-27    ZWS   21.226
2437  2012-04-26    ZWS   21.213

[5165140 rows x 3 columns]


In [8]:
#df.to_csv("ema_5.csv", index=False)

In [9]:
id_list = []
for idx, row in df.iterrows():
    symbol = row["symbol"]
    date = row["date"]
    unique_id = date + '-' + symbol
    id_list.append(unique_id)


In [10]:
print(id_list[1])

2021-12-30-A


In [11]:
df.insert(0, "id", id_list)

In [12]:
print(df)

                  id        date symbol   wma_20
0       2021-12-31-A  2021-12-31      A  156.351
1       2021-12-30-A  2021-12-30      A  155.890
2       2021-12-29-A  2021-12-29      A  155.251
3       2021-12-28-A  2021-12-28      A  154.587
4       2021-12-27-A  2021-12-27      A  154.036
...              ...         ...    ...      ...
2433  2012-05-02-ZWS  2012-05-02    ZWS   21.352
2434  2012-05-01-ZWS  2012-05-01    ZWS   21.313
2435  2012-04-30-ZWS  2012-04-30    ZWS   21.276
2436  2012-04-27-ZWS  2012-04-27    ZWS   21.226
2437  2012-04-26-ZWS  2012-04-26    ZWS   21.213

[5165140 rows x 4 columns]


In [13]:
main = pd.read_csv("main.csv")
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75578 entries, 0 to 75577
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                75578 non-null  object 
 1   date              75578 non-null  object 
 2   symbol            75578 non-null  object 
 3   eps               75578 non-null  float64
 4   epsEstimated      75578 non-null  float64
 5   time              75578 non-null  object 
 6   open              75578 non-null  float64
 7   high              75578 non-null  float64
 8   low               75578 non-null  float64
 9   close             75578 non-null  float64
 10  adjClose          75578 non-null  float64
 11  volume            75578 non-null  int64  
 12  unadjustedVolume  75578 non-null  int64  
 13  change            75578 non-null  float64
 14  changePercent     75578 non-null  float64
 15  vwap              75578 non-null  float64
 16  changeOverTime    75578 non-null  float6

In [14]:
"""id_list = []
format_dates = pd.to_datetime(main["date"]).dt.date
for idx, row in main.iterrows():
    print(idx)
    symbol = row["symbol"]
    date = str(format_dates[idx])
    unique_id = date + '-' + symbol
    id_list.append(unique_id)"""

'id_list = []\nformat_dates = pd.to_datetime(main["date"]).dt.date\nfor idx, row in main.iterrows():\n    print(idx)\n    symbol = row["symbol"]\n    date = str(format_dates[idx])\n    unique_id = date + \'-\' + symbol\n    id_list.append(unique_id)'

In [15]:
#earnings_pricing.insert(0, "id", id_list)

In [16]:
#print(earnings_pricing.info())

In [17]:
print(main.iloc[0])

id                  2021-11-22-A
date                    11/22/21
symbol                         A
eps                         1.21
epsEstimated                1.17
time                         amc
open                       165.0
high                  165.679993
low                   162.779999
close                 162.779999
adjClose              162.565887
volume                   2110400
unadjustedVolume         2110400
change                     -2.22
changePercent             -1.345
vwap                   163.74666
changeOverTime          -0.01345
sma_5                    162.222
sma_10                   160.661
sma_20                   158.968
ema_5                      162.1
ema_10                   160.973
ema_20                    159.72
volatility_5                1.11
volatility_10              3.523
rsi_14                    57.805
wma_5                    162.591
wma_10                   161.426
Name: 0, dtype: object


In [18]:
merged_df = main.merge(df, how="inner", on="id")

In [19]:
print(merged_df)

                   id    date_x symbol_x   eps  epsEstimated time        open  \
0        2021-11-22-A  11/22/21        A  1.21          1.17  amc  165.000000   
1        2021-08-17-A   8/17/21        A  1.10          0.99  amc  161.729996   
2        2021-05-25-A   5/25/21        A  0.71          0.83  amc  133.410004   
3        2021-02-16-A   2/16/21        A  0.94          0.89  amc  129.309998   
4        2020-11-23-A  11/23/20        A  0.72          0.93  amc  111.400002   
...               ...       ...      ...   ...           ...  ...         ...   
75573  2019-03-21-ZUO   3/21/19      ZUO -0.11         -0.11  amc   23.820000   
75574  2018-11-29-ZUO  11/29/18      ZUO -0.10         -0.13  amc   20.240000   
75575  2018-08-30-ZUO   8/30/18      ZUO -0.13         -0.16  amc   33.419998   
75576  2018-05-31-ZUO   5/31/18      ZUO -0.32         -0.40  amc   21.840000   
75577  2021-10-26-ZWS  10/26/21      ZWS  0.55          0.26  bmo   36.490000   

             high         l

In [20]:
# Remove date_y, symbol_y and rename date_x, symbol_x to date & symbol
merged_df = merged_df.drop(['date_y', 'symbol_y'], axis = 1)
merged_df = merged_df.rename(columns={"symbol_x": "symbol", "date_x": "date"})

In [21]:
merged_df.to_csv("main2.csv", index=False)